<a href="https://colab.research.google.com/github/indiegolab/nlp_class/blob/master/rnn_hello(191206).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#tensorflow 2.0 임포트
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [2]:
# Tensorflow 버전 확인
tf.__version__

'2.0.0'

In [3]:
import numpy as np
import tensorflow as tf

idx2char = ['토', '마', '를', '먹', '자']

# 토마토 가르치기 실습: 토토마를자먹 -> 토마토를먹자
# x_data = [[0, 0, 1, 2, 4, 3]]   ## 토토마를자먹
y_data = [[0, 1, 0, 2, 3, 4]]    # 토마토를먹자

num_classes = 5
input_dim = 5  # one-hot size, one_hot을 직접 예측하기위한 hidden_size와 동일
sequence_length = 6  # |토마토를먹자| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],   # 토 0
                       [1, 0, 0, 0, 0],   # 토 0
                       [0, 1, 0, 0, 0],   # 마 1
                       [0, 0, 1, 0, 0],   # 를 2
                       [0, 0, 0, 0, 1],   # 자 4
                       [0, 0, 0, 1, 0]]], # 먹 3
                     dtype=np.float32)

# 클래스 벡터(정수)를 이진 클래스 행렬로 변환합니다. 
## y : 행렬로 변환 할 클래스 벡터 (0에서 num_classes의 정수까지)
## num_classes : 총 클래스 수.
y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print("x_one_hot의 shape", x_one_hot.shape)
print("y_one_hot의 shape", y_one_hot.shape)

x_one_hot의 shape (1, 6, 5)
y_one_hot의 shape (1, 6, 5)



* Sequential 모델  

케라스에서는 층(layer)을 조합하여 모델(model)을 만든다. 모델은 일반적으로 층들의 그래프이다. 가장 흔한 모델 구조는 층을 차례대로 쌓은 tf.keras.Sequential 모델이다.


레이어 생성 자체를 GRU나 LSTM로 생성할 수 도 있고, Cell만 따로 만들어서 RNN 레이어에 넣을 수도 있다.
본 실습에서는 cell만 만들어서 **SimpleRNNCell, LSTMCell, GRUCell**을 구현해본다.

In [4]:
tf.model = tf.keras.Sequential()
'''
셀을 만들고 RNN 레이어에 추가
input_shape = (1,6,5) => 시퀀스 수 (배치), 시퀀스 길이, dim 사이즈
'''
# SimpleRNNCell을 실행
## Argument = 유닛 수, input_shape= 시퀀스 길이, 인풋 차원 수
cell = tf.keras.layers.SimpleRNNCell(units=num_classes, input_shape=(sequence_length, input_dim))
# cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
# cell = tf.keras.layers.GRUCell(units=num_classes, input_shape=(sequence_length, input_dim))

print(num_classes, sequence_length, input_dim)

tf.model.add(tf.keras.layers.RNN(cell=cell, # cell 종류만 바꿔서 실험할 수 있게
                                 return_sequences=True))

# Fully Connected layer, 활성함수 = softmax
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

5 6 5


* tf.keras.Model.compile

tf.keras.Model.compile에는 세 개의 중요한 매개변수가 있다:

    - optimizer: 훈련 과정을 설정. 'adam'이나 'sgd'와 같이 지정할 수도 있다.
    - loss: 최적화 과정에서 최소화될 손실 함수(loss function)를 설정. 평균 제곱 오차(mse)와 categorical_crossentropy, binary_crossentropy 등이 자주 사용된다.
    - metrics: 훈련을 모니터링하기 위해 사용된다. 이름이나 tf.keras.metrics 모듈 아래의 호출 가능한 객체이다. 추가적으로 모델의 훈련과 평가를 즉시 실행하려면 run_eagerly=True 매개변수를 전달할 수 있다.

In [0]:
# 손실함수 = 'categorical_crossentropy, Adam 옵티마이저 사용
tf.model.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

* tf.keras.Model.fit: 모델 훈련  

tf.keras.Model.fit에는 세 개의 중요한 매개변수가 있다:
// e.g. model.fit(data, labels, epochs=10, batch_size=32)

    - epochs: 훈련은 에포크(epoch)로 구성된다. 한 에포크는 전체 입력 데이터를 한번 순회하는 것이다(작은 배치로 나누어 수행된다).
    - batch_size: 넘파이 데이터를 전달하면 모델은 데이터를 작은 배치로 나누고 훈련 과정에서 이 배치를 순회한다. 이 정수 값은 배치의 크기를 지정한다.전체 샘플 개수가 배치 크기로 나누어 떨어지지 않으면 마지막 배치의 크기는 더 작을 수 있다.
    - validation_data: 모델의 프로토타입(prototype)을 만들 때는 검증 데이터(validation data)에서 간편하게 성능을 모니터링해야 한다. 입력과 레이블(label)의 튜플을 이 매개변수로 전달하면 에포크가 끝날 때마다 추론 모드(inference mode)에서 전달된 데이터의 손실과 측정 지표를 출력한다.

In [6]:
# 훈련
tf.model.fit(x_one_hot, y_one_hot, epochs=50) # 에폭 = 50
tf.model.summary()

predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)

    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

Train on 1 samples
Epoch 1/50
1/1 [==============================] - 2s 2s/sample - loss: 1.7291 - accuracy: 0.1667
Epoch 2/50
1/1 [==============================] - 0s 18ms/sample - loss: 1.3194 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 12ms/sample - loss: 0.9917 - accuracy: 0.6667
Epoch 4/50
1/1 [==============================] - 0s 13ms/sample - loss: 0.7853 - accuracy: 0.8333
Epoch 5/50
1/1 [==============================] - 0s 11ms/sample - loss: 0.6303 - accuracy: 0.8333
Epoch 6/50
1/1 [==============================] - 0s 11ms/sample - loss: 0.4985 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 13ms/sample - loss: 0.3865 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 10ms/sample - loss: 0.2961 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 13ms/sample - loss: 0.2270 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 15ms/sample - loss: 0.1755 - accuracy: